In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import copy
from sklearn.model_selection import train_test_split
from datetime import datetime

2024-04-30 12:34:22.675681: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-30 12:34:23.142698: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
BATCH_SIZE = 512

learning_rate = 0.001

In [3]:
AS_dataset = pd.read_csv('./hairpin_filter_dataset.csv', encoding='utf-8')

In [4]:
selected_AS_dataset = AS_dataset[(AS_dataset['freq'] >= 2.5) & (AS_dataset['freq'] <= 6.5)].reset_index(drop=True)

In [5]:
odd_X = selected_AS_dataset.loc[::2, 'freq':'S1'].to_numpy(dtype = np.float32)
odd_y = selected_AS_dataset.loc[::2, 'S21r':'S21i'].to_numpy(dtype = np.float32)

In [6]:
even_X = selected_AS_dataset.loc[1::2, 'freq':'S1'].to_numpy(dtype = np.float32)
even_y = selected_AS_dataset.loc[1::2, 'S21r':'S21i'].to_numpy(dtype = np.float32)

In [7]:
odd_X[:, 1] = (odd_X[:, 1] - 200) / (2000 - 200)
odd_X[:, 2] = (odd_X[:, 2] - 200) / (2000 - 200)
odd_X[:, 3] = (odd_X[:, 3] - 200) / (2000 - 200)
odd_X[:, 4] = (odd_X[:, 4] - 200) / (4500 - 200)
odd_X[:, 5] = (odd_X[:, 5] - 200) / (4500 - 200)
odd_X[:, 6] = (odd_X[:, 6] - 200) / (4500 - 200)
odd_X[:, 7] = (odd_X[:, 7] - 200) / (4500 - 200)
odd_X[:, 8] = (odd_X[:, 8] - 9000) / (12000 - 9000)
odd_X[:, 9] = (odd_X[:, 9] - 100) / (300 - 100)

In [8]:
even_X[:, 1] = (even_X[:, 1] - 200) / (2000 - 200)
even_X[:, 2] = (even_X[:, 2] - 200) / (2000 - 200)
even_X[:, 3] = (even_X[:, 3] - 200) / (2000 - 200)
even_X[:, 4] = (even_X[:, 4] - 200) / (4500 - 200)
even_X[:, 5] = (even_X[:, 5] - 200) / (4500 - 200)
even_X[:, 6] = (even_X[:, 6] - 200) / (4500 - 200)
even_X[:, 7] = (even_X[:, 7] - 200) / (4500 - 200)
even_X[:, 8] = (even_X[:, 8] - 9000) / (12000 - 9000)
even_X[:, 9] = (even_X[:, 9] - 100) / (300 - 100)

In [9]:
odd_X_train, odd_X_vali, odd_y_train, odd_y_vali = train_test_split(odd_X, odd_y, test_size=0.05, random_state=0)

In [10]:
even_X_train, even_X_vali, even_y_train, even_y_vali = train_test_split(even_X, even_y, test_size=0.05, random_state=0)

In [11]:
odd_dataset_train = tf.data.Dataset.from_tensor_slices((odd_X_train, odd_y_train))
odd_dataset_train = odd_dataset_train.shuffle(buffer_size=odd_X_train.shape[0])
odd_dataset_train = odd_dataset_train.batch(BATCH_SIZE)
odd_dataset_train = odd_dataset_train.prefetch(tf.data.experimental.AUTOTUNE)

2024-04-30 12:34:25.515572: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9604 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:17:00.0, compute capability: 7.5
2024-04-30 12:34:25.516170: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 9621 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:65:00.0, compute capability: 7.5


In [12]:
even_dataset_train = tf.data.Dataset.from_tensor_slices((even_X_train, even_y_train))
even_dataset_train = even_dataset_train.shuffle(buffer_size=even_X_train.shape[0])
even_dataset_train = even_dataset_train.batch(BATCH_SIZE)
even_dataset_train = even_dataset_train.prefetch(tf.data.experimental.AUTOTUNE)

In [13]:
X_vali = np.concatenate([odd_X_vali, even_X_vali])
y_vali = np.concatenate([odd_y_vali, even_y_vali])

In [14]:
@tf.keras.saving.register_keras_serializable()
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.dense1 = tf.keras.layers.Dense(units=512, activation='leaky_relu')
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.dense2 = tf.keras.layers.Dense(units=512, activation='leaky_relu')
        self.dense3 = tf.keras.layers.Dense(units=128, activation='leaky_relu')
        self.dense4 = tf.keras.layers.Dense(units=160, activation='leaky_relu')
        self.dense5 = tf.keras.layers.Dense(units=2)

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.bn1(x)
        x = self.dense2(x)
        x = self.dense3(x)
        x = self.dense4(x)
        output = self.dense5(x)
        return output

In [15]:
class ParaServer:
    def __init__(self):
        self.model = MLP()
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    def upload(self, grads):
        self.optimizer.apply_gradients(grads_and_vars=zip(grads, self.model.trainable_variables))
        return self.model
    def download(self):
        return self.model
    def initModel(self, x):
        self.model(x)

In [16]:
def valiAll():
    m = ps.download()
    model = copy.deepcopy(m)
    
    y_v_p = model(X_vali, training=False)
    va_mse = tf.reduce_mean(tf.square(y_v_p - y_vali))
    va_r2 = 1 - tf.reduce_sum(tf.square(y_v_p - y_vali)) / tf.reduce_sum(tf.square(y_vali - tf.reduce_mean(y_vali)))

    odd_y_v_p = model(odd_X_vali, training=False)
    odd_va_mse = tf.reduce_mean(tf.square(odd_y_v_p - odd_y_vali))
    odd_va_r2 = 1 - tf.reduce_sum(tf.square(odd_y_v_p - odd_y_vali)) / tf.reduce_sum(tf.square(odd_y_vali - tf.reduce_mean(odd_y_vali)))

    even_y_v_p = model(even_X_vali, training=False)
    even_va_mse = tf.reduce_mean(tf.square(even_y_v_p - even_y_vali))
    even_va_r2 = 1 - tf.reduce_sum(tf.square(even_y_v_p - even_y_vali)) / tf.reduce_sum(tf.square(even_y_vali - tf.reduce_mean(even_y_vali)))
    
    print("vali mse:{} r2:{}".format(va_mse, va_r2))
    print("odd vali mse:{} r2:{}".format(odd_va_mse, odd_va_r2))
    print("even vali mse:{} r2:{}".format(even_va_mse, even_va_r2))
    r2sv.append(va_r2.numpy())

In [17]:
class Node:
    def __init__(self, dataset, freq):
        self.model = MLP()
        self.freq = freq
        self.dataset_train = dataset
    def train(self, num_epochs):
        m = ps.download()
        self.model = copy.deepcopy(m)
        for epoch_index in range(num_epochs):
            epoch_mse_avg = tf.keras.metrics.Mean()
            epoch_r2_avg = tf.keras.metrics.Mean()
            for X, y in self.dataset_train:
                with tf.GradientTape() as tape:
                    y_pred = self.model(X, training=True)
                    tr_mse = tf.reduce_mean(tf.square(y_pred - y))
                grads = tape.gradient(tr_mse, self.model.trainable_variables)
                m = ps.upload(grads)
                self.model = copy.deepcopy(m)
                tr_r2 = 1 - tf.reduce_sum(tf.square(y_pred - y)) / tf.reduce_sum(tf.square(y - tf.reduce_mean(y)))
                epoch_mse_avg(tr_mse)
                epoch_r2_avg(tr_r2)
                # if epoch_index in np.arange(0, num_epochs, 25).tolist() or epoch_index == num_epochs - 1:
            print("node:{} epoch:{}".format(self.freq, epoch_index))
            print("train mse:{} r2:{}".format(epoch_mse_avg.result(), epoch_r2_avg.result()))
            r2s.append(tr_r2.numpy())
            valiAll()

In [18]:
r2s = []
r2sv = []

In [19]:
ps = ParaServer()
ps.initModel(X_vali)

In [20]:
nodeList = [Node(odd_dataset_train, 'odd'), Node(even_dataset_train, 'even')]

In [21]:
nodeList[0].train(50)
nodeList[1].train(50)

nodeList[0].train(50)
nodeList[1].train(50)

2024-04-30 12:34:26.544797: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5606e93c33b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-04-30 12:34:26.544816: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
2024-04-30 12:34:26.544820: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (1): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
2024-04-30 12:34:26.547902: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-04-30 12:34:26.652969: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2024-04-30 12:34:26.761531: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


node:odd epoch:0
train mse:0.09092696011066437 r2:0.05030122771859169
mse:0.0956863984465599 r2:-0.0011771917343139648
node:odd epoch:1
train mse:0.05064026638865471 r2:0.4711219072341919
mse:0.09660656005144119 r2:-0.010804891586303711
node:odd epoch:2
train mse:0.040342289954423904 r2:0.578755795955658
mse:0.0956355482339859 r2:-0.0006451606750488281
node:odd epoch:3
train mse:0.04049395024776459 r2:0.5753185153007507
mse:0.09557974338531494 r2:-6.127357482910156e-05
node:odd epoch:4
train mse:0.03795267269015312 r2:0.6050366759300232
mse:0.09638429433107376 r2:-0.00847935676574707
node:odd epoch:5
train mse:0.03594781830906868 r2:0.6246951222419739
mse:0.09660671651363373 r2:-0.010806560516357422
node:odd epoch:6
train mse:0.03711216524243355 r2:0.6119914054870605
mse:0.09616044163703918 r2:-0.00613713264465332
node:odd epoch:7
train mse:0.035107970237731934 r2:0.6331291794776917
mse:0.09602080285549164 r2:-0.004676103591918945
node:odd epoch:8
train mse:0.03567523881793022 r2:0.627